In [1]:
import os
os.getcwd()


'c:\\github\\samratkar.github.io\\_posts\\concepts\\genai\\notes-codes\\aeroslm'

In [2]:
import torch
import aeroslm
from huggingface_hub import hf_hub_download

pt_path = hf_hub_download(
    repo_id="samratkar/slm_tinystories",
    filename="slm_tinystories.pt",
)
# Load the model
model = aeroslm.GPT()
state_dict = torch.load(pt_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()  # Set to evaluation mode


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 384)
    (wpe): Embedding(128, 384)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=True)
          (c_proj): Linear(in_features=384, out_features=384, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=50257, bias=False)
)

In [3]:
model.infer("Once upon a time there was a pumpkin.")

"Once upon a time there was a pumpkin. Inside the field, there was a family. One day, they decided to wear baby, who was walking in the park.\n\nWhen they decided to go to the beach, it was cold and cold. They buckled hands and stirred the water and started to jump into the water.\n\nWhen they got home, they saw a big fountain in the sink. Jim wanted to get back but he couldn't wait. Then, the dragon swam back inside, when he stopped. When he got there, he saw something exciting. It was a busy man with a white nose. The man lay down on the more and hear them crawl from Tim's pocket.Ben and his mom were in a long city. Ben liked the fields and beautiful animals. They watched them go deep and usually got very slowacked.\n\nOne day, they decided to go to a school. Ben went to the jungle with his mom. He saw many kids playing a game of rocks. Ben thinks they are"

In [1]:
import tiktoken
import torch
import aeroslm

enc = tiktoken.get_encoding("gpt2")
context = torch.tensor(enc.encode_ordinary("There was an ai agent")).unsqueeze(dim=0)
model = aeroslm.GPT()
# Trace the model's forward pass
traced_model = torch.jit.trace(model, context)
traced_model.save("slm_tinystories_scripted.pt")

RuntimeError: 
Module 'CausalSelfAttention' has no attribute 'bias' :
  File "c:\github\samratkar.github.io\_posts\concepts\genai\notes-codes\aeroslm\aeroslm.py", line 46
        else:
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
                                  ~~~~~~~~~ <--- HERE
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
'CausalSelfAttention.forward' is being compiled since it was called from 'Block.forward'
  File "c:\github\samratkar.github.io\_posts\concepts\genai\notes-codes\aeroslm\aeroslm.py", line 73
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
                                   ~ <--- HERE
        x = x + self.mlp(self.ln2(x))
        return x
'Block.forward' is being compiled since it was called from 'GPT.forward'
  File "c:\github\samratkar.github.io\_posts\concepts\genai\notes-codes\aeroslm\aeroslm.py", l